In [ ]:
# adapting function for XLSX from public records request #23-51

import pandas as pd

fn = 'CulverCity_RentalRegistry_PublicRecordsRequest.xlsx'
adapt = pd.read_excel(fn)
adapt = adapt.rename(columns={
    'Property Address':'AddressLine1',
    'Property City':'AddressLine2',
    'Owner Name':'Owner',
    'Owner Email':'Email',
    'CertOfOccuDate':'OccDate',
    'YearBuilt':'YearBuilt',
    'UnitNumb':'UnitNumb',
    'UnitStatusDescription':'Unit Status',
    'NumofBeds':'NumofBeds',
    'NumOfBaths':'NumOfBaths',
    'Section8Tenant':'Section8',
    'EmcumberedUnit':'EmcumberedUnit',
    'ReportDate':'PastRentReportDate',
    'Amount':'RentAmount'})

adapt['OccDate'] = adapt['OccDate'].replace("<N/A>",None)
adapt['OccDate'] = pd.to_datetime(adapt['OccDate'])

adapt['PastRentReportDate'] = adapt['PastRentReportDate'].replace("<N/A>",None)
adapt['PastRentReportDate'] = pd.to_datetime(adapt['PastRentReportDate'])

adapt['RentAmount'] = adapt['RentAmount'].replace("<N/A>",None)
adapt['RentAmount'] = adapt['RentAmount'].replace('[\$,]', '', regex=True).astype(float)
adapt['RentAmount'] = pd.to_numeric(adapt['RentAmount'])

adapt['EmcumberedUnit'] = adapt['EmcumberedUnit'].replace({"YES":"Yes","NO":"No"})
adapt['Section8'] = adapt['Section8'].replace({"YES":"Yes","NO":"No"})

adapt['Unit Status'] = adapt['Unit Status'].replace("Section 8","Rented")

adapt['Owner'] = adapt['Owner'].replace("<N/A>","")
adapt['Email'] = adapt['Email'].replace("<N/A>","")

adapt.to_excel('Rent Registry_PRA 23-51.xlsx')

In [45]:
# run this to read in the rent registry XLSX file

import pandas as pd
import numpy as np
import usaddress
import datetime

def cleanOwnerName(s):
    if pd.isna(s):
        return ''
    return s.removeprefix(',').strip()

def cleanAddressLine1(a):
    if pd.isna(a):
        return ''
    if isinstance(a,str) and '10144 CULVER CITY' in a:
        return '10144 CULVER BLVD'
    if isinstance(a,datetime.datetime):
        return '4051 JACKSON AVE'
    if isinstance(a,str) and '3959 AND 3961 BENTLEY AVE' in a:
        return '3959-3961 BENTLEY AVE'
    if isinstance(a,str) and '4222 & 4224 JASMINE AVE' in a:
        return '4222-4224 JASMINE AVE'
    if isinstance(a,str) and '4070 LAFAYETTE PL # House B' in a:
        return '4070 LAFAYETTE PL House B'
    if isinstance(a,str) and '4350-68 OVERLOAD AVE' in a:
        return '4350-68 OVERLAND AVE'
    if isinstance(a,str) and 'Do not own a rental property in Culver City' in a:
        return ''
    tagged_address, address_type= usaddress.tag(str(a))
    if 'StreetNamePostType' not in tagged_address.keys():
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    else:
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])

fn = 'Rent Registry_PRA 22-496.xlsx'
fn = 'Rent Registry_PRA 23-51.xlsx'
#fn = 'test.xlsx'
ccrr = pd.read_excel(fn)
#ccrr['AddressLine1'] = ccrr['AddressLine1'].str.replace('10144 CULVER CITY','10144 CULVER BLVD')
ccrr['AddressLine1'] = ccrr['AddressLine1'].apply(cleanAddressLine1)
ccrr['Owner'] = ccrr['Owner'].apply(cleanOwnerName)
ccrr = ccrr.sort_values(by='PastRentReportDate',ascending=False).reset_index()

In [ ]:
# geocode addresses in ccrr dataframe

from geocodio import GeocodioClient
import time

key = 'd8f821033f22f02a2d80862a2286d6d2803dd2a'
client = GeocodioClient(key)

# run this to geocode all the addresses in the rent registry
# make sure you use the exact same usaddress format as below in the "address" variable
existing_geocode = pd.read_csv('cc_geocoded.csv')
addresses = ccrr.drop_duplicates(['AddressLine1'])['AddressLine1']
#addresses = addresses.apply(cleanAddressLine1).drop_duplicates()
headers = ['address','lat','lon']
geocoded = existing_geocode.copy()
l = 0
addresses_to_geocode = []
for address in addresses:
    if address == '':
        continue
    elif address in existing_geocode['address'].values:
        continue
    addresses_to_geocode.append(address)

    l += 1
    if l > 10000:
        break

if len(addresses_to_geocode) > 0:
    geocoded_location = client.batch_geocode([s + ', CULVER CITY, CA' for s in addresses_to_geocode])
    coords = geocoded_location.coords
    #coords = []
    #for address in addresses_to_geocode:
    #    lat = l*20
    #    lon = l*2+3
    #    l += 1
    #    coords.append((lat,lon))
    #print(coords)

    i = 0
    for address in addresses_to_geocode:
        if coords[i] == None:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, '', '']],columns=headers)])
        else:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, coords[i][0], coords[i][1]]],columns=headers)])
        i += 1

    existing_geocode.to_csv('cc_geocoded_backup-' + time.strftime("%Y%m%d-%H%M%S") + '.csv',index=False)
    geocoded.to_csv('cc_geocoded.csv',index=False)
    

In [98]:
# THIS ONE CREATES THE FLAT GEOJSON

import datetime
import time
import orjson
import math
from slugify import slugify

flat_json = dict()
flat_json['type'] = 'FeatureCollection'
flat_features = []
a = []

existing_geocode = pd.read_csv('cc_geocoded.csv')
registered_addresses = []
owners = dict()

for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb']).iterrows():
    if pd.isna(row['AddressLine1']):
        continue

    line1 = row['AddressLine1']
    line2 = row['AddressLine2']
    unit = row['UnitNumb']
    beds = row['NumofBeds']
    baths = row['NumOfBaths']
    built = row['YearBuilt']
    rent = row['RentAmount']
    if rent == "<N/A>":
        rent = ""
    if pd.isna(row['PastRentReportDate']):
        rentdate = ''
    elif isinstance(row['PastRentReportDate'],str):
        rentdate = row['PastRentReportDate']
    else:
        rentdate = row['PastRentReportDate'].strftime('%Y/%m/%d')
    encumbered = row['EmcumberedUnit']
    status = row['Unit Status']

    if pd.isna(row["Owner"]):
        owner = ''
    else:
        owner = row['Owner'].removeprefix(', ').removeprefix(',').title()

    if pd.isna(row['Email']):
        email = ''
    else:
        email = row['Email']
    occdate = row['OccDate']
    section = row['Section8']
    
    address = line1
    if address == '':
        continue
    
    #create flat_json with a feature for every unit
    flat = dict()
    flat['type'] = 'Feature'

    geo = existing_geocode[existing_geocode['address'] == address]
    flat['geometry'] = {
        'type':'Point',
        'coordinates':[geo['lon'].values[0].item(),geo['lat'].values[0].item()]
    }

    slug = slugify(address)
    address = address.title()
    flat['properties'] = {
        'address':address,
        'unit':unit,
        'beds':beds,
        'baths':baths,
        'built':built,
        'rent':rent,
        'rentdate':rentdate,
        'encumbered':encumbered,
        'status':status,
        'slug':slug,
        'owner':owner,
        'email':email,
        #'occdate':occdate,
        'section':section
    }

    if slug not in owners.keys():
        owners[slug] = []
    if owner not in owners[slug]:
        owners[slug].append(owner)

    registered = False
    if rent == '':
        registered = False
    else:
        registered = True
    #for key in flat['properties'].keys():
    #    if flat['properties']['address'] == '11460 WASHINGTON BLVD':
    #        a.append(flat['properties'][key])
    #    if key == 'address' or key == 'slug':
    #        continue
    #    if flat['properties'][key] != None and flat['properties'][key] != '' and flat['properties'][key] != 'null' and (isinstance(flat['properties'][key],str) or isinstance(flat['properties'][key],datetime.datetime) or not math.isnan(flat['properties'][key])):
    #        registered = True
    #        break
    flat['properties']['registered'] = registered
    if registered:
        registered_addresses.append(address)
    flat_features.append(flat)

# if an address is clearly registered, remove any rows for that address that are not registered!!! (e.g., 3801 College)
flat_features = [x for x in flat_features if not(x['properties']['address'] in registered_addresses and not x['properties']['registered'])]

for i in range(len(flat_features)):
    if len(owners[flat_features[i]['properties']['slug']]) > 1:
        flat_features[i]['properties']['multiple_owners'] = True
    else:
        flat_features[i]['properties']['multiple_owners'] = False

flat_json['features'] = flat_features

with open("ccrr-flat-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(flat_json))

In [99]:
# from ccrr dataframe, create list of owners sorted by number of occurrences

owners = ccrr['Owner'].replace('', np.nan).dropna()
owners.value_counts().reset_index(name='count').sort_values(['count'], ascending=False).to_csv('owner_counts.csv')

In [100]:
# turn flat_json into csv file of properties

headers = ['address',
        'unit',
        'beds',
        'baths',
        'built',
        'rent',
        'rentdate',
        'encumbered',
        'status',
        'registered',
        'slug',
        'owner',
        'email',
        #'occdate',
        'section',
        'multiple_owners']
flat_df = pd.DataFrame(columns=headers)


for f in flat_json['features']:
    row = []
    for i in range(len(headers)):
        row.append(f['properties'][headers[i]])
    flat_df = pd.concat([flat_df,pd.DataFrame([row],columns=headers)])
flat_df.to_csv("ccrr-flat-" + time.strftime("%Y%m%d-%H%M%S") + ".csv")

In [91]:
multiple_owners = dict()
blank_and_multiple = dict()
for slug in flat_df["slug"].drop_duplicates():
    units = flat_df[flat_df["slug"] == slug]
    owners = list(units['owner'].drop_duplicates())
    if '' in owners and len(owners) > 2:
        blank_and_multiple[slug] = owners
    if len(owners) > 1:
        multiple_owners[slug] = owners

In [96]:
len(multiple_owners)

132

In [84]:
len(multiple_owners)

132

In [ ]:
# turn big PDF data into 5 different PDFs, each representing a different set of columns

from PyPDF2 import PdfWriter, PdfReader

inputpdf = PdfReader(open("Rent Registry_PRA 23-51.pdf", "rb"))
pagesper = 452
tables = 5

for i in range(tables):
    output = PdfWriter()
    for j in range(pagesper):
        output.add_page(inputpdf.pages[i*pagesper+j])
    with open("document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)

In [ ]:
# check if those 5 PDFs all have the same number \n "rows"
import PyPDF2

fns = [
    'document-page0.pdf',
    'document-page1.pdf',
    'document-page2.pdf',
    'document-page3.pdf',
    'document-page4.pdf',
    ]
for fn in fns:
    pdfFileObj = open(fn ,'rb')

    pdfReader = PyPDF2.PdfReader(pdfFileObj)

    pages = len(pdfReader.pages)


    new_lines = 0
    for i in range(pages):
        new_lines += pdfReader.pages[i].extract_text().count('\n')+1
    print(fn,": ",new_lines)
    pdfFileObj.close()

In [ ]:
# take the 5 PDFs and split the newlines, making them into rows

import PyPDF2
import pandas as pd

fns = [
    'document-page0.pdf',
    'document-page1.pdf',
    'document-page2.pdf',
    'document-page3.pdf',
    'document-page4.pdf',
    ]

columns = []

# page_ones = []
for fn in fns:
    col = []
    pdfFileObj = open(fn ,'rb')

    pdfReader = PyPDF2.PdfReader(pdfFileObj)
    for page in pdfReader.pages:
        col += page.extract_text().split('\n')

    # page_ones.append(pdfReader.pages[0].extract_text())
    columns.append(col)
    pdfFileObj.close()

In [ ]:
# take the 2d array created above and change to a pandas df, using regexes to split the columns

# you can run the main functions after this to import the XLSX file and generate GEOJSON
pdf_units = pd.DataFrame()
for col in columns:
    pdf_units[col[0]] = col[1:]

reg_strings = {
    columns[0][0]:r'(?P<AddressLine1>.+)',
    columns[1][0]:r'(?P<Owner>.+)',
    columns[2][0]:r'(?P<Email>\S+\@\S+)?\s*(?P<OccDate>\d?\d/\d?\d/\d{4}|<N/A>|\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}\.\d{3})\s*(?P<YearBuilt>\d{4}|0)',
    columns[3][0]:r'(?P<UnitNumb>.*)\s*(?P<UnitStatus>Rented|Vacant|Section 8*)\s*(?P<NumofBeds>\d*)\s*(?P<NumOfBaths>\S*)',
    columns[4][0]:r'(?P<Section8>YES|NO*)\s*(?P<EmcumberedUnit>YES|NO*)\s*(?P<PastRentReportDate>\S*)\s*(?P<RentAmount>\S*)'
}

pdf_units[columns[3][0]].str.extract(reg_strings[columns[3][0]]).loc[[3666]]

In [ ]:
# take pdf_units and convert to dataframe, then save as XLSX file


split_units = pd.DataFrame()
for i in range(5):
    split_units = pd.concat([split_units,pdf_units[columns[i][0]].str.extract(reg_strings[columns[i][0]],expand=True)], axis=1)

split_units = split_units.rename(columns={'UnitStatus':'Unit Status'})

split_units['AddressLine2'] = ""

split_units['OccDate'] = split_units['OccDate'].replace("<N/A>",None)
split_units['OccDate'] = pd.to_datetime(split_units['OccDate'])

split_units['PastRentReportDate'] = split_units['PastRentReportDate'].replace("<N/A>",None)
split_units['PastRentReportDate'] = pd.to_datetime(split_units['PastRentReportDate'])

split_units['RentAmount'] = split_units['RentAmount'].replace("<N/A>",None)
split_units['RentAmount'] = split_units['RentAmount'].replace('[\$,]', '', regex=True).astype(float)
split_units['RentAmount'] = pd.to_numeric(split_units['RentAmount'])

split_units['EmcumberedUnit'] = split_units['EmcumberedUnit'].replace({"YES":"Yes","NO":"No"})
split_units['Section8'] = split_units['Section8'].replace({"YES":"Yes","NO":"No"})

split_units['Unit Status'] = split_units['Unit Status'].replace("Section 8","Rented")

split_units['Owner'] = split_units['Owner'].replace("<N/A>","")

split_units.to_excel('Rent Registry_PRA 23-51.xlsx')